In [1]:
from typing import *

from ortools.init import pywrapinit
from ortools.linear_solver import pywraplp

In [2]:
def gen_substring_set(s: str) -> Set[str]:
  res: Set[str] = set()

  for i in range(len(s)+1):
    for j in range(i+1,len(s)+1):
      res.add(s[i:j])

  return res

In [3]:
def gen_common_blocks(S: List[str], T: Set[str]):
  blocks = {}
  res: Dict[str, List[Tuple[int,int]]] = {}

  for s_idx, s in enumerate(S):
    for i in range(len(s)+1):
      for j in range(i+1,len(s)+1):
        if s[i:j] in T:
          if (s[i:j],s_idx) in blocks:
            blocks[s[i:j], s_idx].append(i)
          else:
            blocks[s[i:j], s_idx] = [i]

  for t in T:
    res[t] = list(zip(blocks[t,0],blocks[t,1]))

  return res

In [4]:
solver = pywraplp.Solver.CreateSolver('GLOP')

In [5]:
S1 = 'AGACTG'
S2 = 'ACTAGG'
N = len(S1)

In [6]:
SUBS_OF_S1 = gen_substring_set(S1)
SUBS_OF_S2 = gen_substring_set(S2)
T = set.intersection(SUBS_OF_S1, SUBS_OF_S2)

In [7]:
B = gen_common_blocks([S1,S2], T)

In [8]:
M = len(B)

In [9]:
B

{'AG': [(0, 3)],
 'T': [(4, 2)],
 'A': [(0, 0), (2, 3)],
 'ACT': [(2, 0)],
 'CT': [(3, 1)],
 'C': [(3, 1)],
 'G': [(1, 4), (5, 5)],
 'AC': [(2, 0)]}

## Variáveis

In [10]:
x = {}
for i,(t,b) in enumerate(B.items()):
  for k in b:
    print(i,k[0],k[1])
    x[i,k[0],k[1]] = solver.BoolVar(f'x_{i}')

0 0 3
1 4 2
2 0 0
2 2 3
3 2 0
4 3 1
5 3 1
6 1 4
6 5 5
7 2 0


## Constraint

caso: $\sum\limits_{i \in \{1,\dots,m\ | k^{1}_{i} \leq j < k^{1}_{i} + |t_i|\}} x_i = 1$

In [11]:
for j in range(N):
  constraint_terms = []
  for i,(t,b) in enumerate(B.items()):
    for k in b:
      if k[0] <= j < (k[0] + len(t)):
        constraint_terms.append(x[i,k[0],k[1]])
  
  solver.Add(solver.Sum(constraint_terms) == 1)

caso: $\sum\limits_{i \in \{1,\dots,m\ | k^{2}_{i} \leq j < k^{2}_{i} + |t_i|\}} x_i = 1$

In [12]:
for j in range(N):
  constraint_terms = []
  for i,(t,b) in enumerate(B.items()):
    for k in b:
      if k[1] <= j < (k[1] + len(t)):
        constraint_terms.append(x[i,k[0],k[1]])
  
  solver.Add(solver.Sum(constraint_terms) == 1)

## Objective function

$ \text{Minimize} \sum\limits^{m}_{i=1} x_i $

In [14]:
objective_terms = []

for i,(t,b) in enumerate(B.items()):
  for k in b:
    objective_terms.append(x[i,k[0],k[1]])

solver.Minimize(solver.Sum(objective_terms))    

## Solving

In [16]:
status = solver.Solve()

In [17]:
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
  print(solver.Objective().Value())
else:
  print("Não há solução ótima.")

3.0


In [20]:
res = []

for i,(t,b) in enumerate(B.items()):
  for k in b:
    if x[i,k[0],k[1]].solution_value() == 1:
      res.append(t)

In [22]:
res.sort()

In [23]:
res

['ACT', 'AG', 'G']